### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [176]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [177]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [178]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [179]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [180]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [181]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [182]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [183]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [184]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [185]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [186]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [187]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [188]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [189]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [190]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [191]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [192]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [193]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [194]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [195]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


# Resolución
### Importar librerías

In [196]:
import random
from sklearn.model_selection import GridSearchCV

### Ejercicio 1 - Vectorización

In [197]:
# Configurar semilla y longitud de texto a imprimir
random.seed(49)
TXT_LEN = 180
ELLIPSIS = "..."

# Obtener 5 documentos aleatorios
random_idx = random.sample(range(X_train.shape[0]), 5)
X_train_subset = X_train[random_idx]

# Calcular array de similaridades
cos_sim_arr = cosine_similarity(X_train_subset, X_train)

# Recorremos la lista de documentos para obtener los otros cinco mas similares
for i, j in enumerate(random_idx):
    # Motramos un fragmento de texto y su tipo
    sample_text = newsgroups_train.data[j][:TXT_LEN].replace("\n", "") + ELLIPSIS
    sample_class = newsgroups_train.target_names[y_train[j]]
    print(f'Texto numero {i+1}: "{sample_text}"')
    print(f"Tipo de documento: {sample_class}")

    # Obtenemos los indices de los documentos más similares (sin contarse a él mismo)
    # Ordenamos, invertimos el orden (mayor a menor) y obtenemos los 5 primeros indices saltandose el primero (mismo documento)
    cos_sim = cos_sim_arr[i]
    sim_indexes = cos_sim.argsort()[::-1][1:6]

    # Mostramos los documentos más similares
    for k,l in enumerate(sim_indexes):
        sim_text = newsgroups_train.data[l][:TXT_LEN].replace("\n", "") + ELLIPSIS
        sim_class = newsgroups_train.target_names[y_train[l]]
        cos_value = cos_sim[l]
        print(f'Documento similar numero {k+1}: "{sim_text}"')
        print(f"Tipo de documento: {sim_class}")
        print(f"Similaridad : {cos_value}\n")
    print("-----------------------\n")

Texto numero 1: "       As perhaps some insight into how this sort of thing works, thelocal college newspaper had a big crusade to have the U.T. policerelease crime stats.  (The school claimed ..."
Tipo de documento: talk.politics.guns
Documento similar numero 1: "Accounts of Anti-Armenian Human Right Violations in Azerbaijan #008 Part B                 Prelude to Current Events in Nagorno-Karabakh				(Part B of #008)      +------------..."
Tipo de documento: talk.politics.mideast
Similaridad : 0.2512904561529162

Documento similar numero 2: "Hello, my friends and I are running the Homewood Fantasy BaseballLeague (pure fantasy baseball teams). Unfortunely, we are running the leagueusing Earl Weaver Baseball II with th..."
Tipo de documento: rec.sport.baseball
Similaridad : 0.25059195814637086

Documento similar numero 3: "Does anybody else think that WS stats should become part ofa player's career stats?   Why not? ..."
Tipo de documento: rec.sport.baseball
Similaridad : 0.246338875732

#### Comentario:
Podemos observar que para los 5 documentos seleccionados aleatoriamente los documentos más similares suelen compartir el mismo tipo de documento, comparando el contenido del texto podemos encontrar coincidencias en las palabras que usan.

Curiosamente, en el primer ejemplo nos encontramos con un fragmento de texto muy breve como tercer documento más similar. Esto seguramente se debe a que la mayoría de palabras de dicho documento también forman parte del "original". Curiosamente se cuelan dos documentos de deportes-baseball como documentos similares a uno de política-armas, posiblemente debido a que en todos ellos hablan de estadísticas (aunque el tema sea completamente distinto). 

### Ejercicio 2 - Entrenamiento
#### Mejorar el modelo multinomial

Vamos a experimentar con el vectorizador para ver si se pueden mejorar la métrica 0.5854345727938506.

Tras probar con varias combinaciones (variando la frecuencia máxima de aparicion en documentos entre un 65% y 85%, el mínimo de apariciones en documentos entre 2 y 10, manteniendo unigramas y digramas en todos los intentos) se obtuvo un vectorizador que mejoró el score en 0.006. Un valor muy mediocre. Sin embargo, al configurar las Stopwords al idioma inglés se consiguió una mejora de más del 6%:

In [198]:
# instanciamos un nuevo vectorizador
new_vect = TfidfVectorizer(max_df=0.85, min_df=2, ngram_range=(1, 2), stop_words='english')
new_X_train = new_vect.fit_transform(newsgroups_train.data)

# Intanciar el modelo multinomial de prueba, se hace fit
test_clf = MultinomialNB()
test_clf.fit(new_X_train, y_train)

# vectorizamos set de prueba y se aplica el predict del modelo de prueba
new_X_test = new_vect.transform(newsgroups_test.data)
y_pred =  test_clf.predict(new_X_test)

f1_score(y_test, y_pred, average='macro')



0.6503941723196313

Con el vectorizador mejorado, ahora vamos a utilizar gridsearch para encontrar la mejor configuración del modelo multinomial.

In [199]:
# Instanciar modelo
new_clf = MultinomialNB()

# Definición del conjunto de hiperparámetros
param_grid = {'alpha': np.arange(0.01, 1, 0.05)}

# Instanciamos y ejecutamos gridSearch
grid_search = GridSearchCV(new_clf, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(new_X_train, y_train)

# Obtenemos el mejor hiperparámetro y reentrenamos el modelo
alpha = grid_search.best_params_['alpha']
print("Valor de alpha más efectivo:", alpha)

new_clf = MultinomialNB(alpha=alpha)
new_clf.fit(new_X_train, y_train)

# Recalculamos el score
y_pred = new_clf.predict(new_X_test)
f1_score(y_test, y_pred, average='macro')


Valor de alpha más efectivo: 0.01


0.6861834299321644

Con los cambios realizados, se obtiene un multinomial con un score superior al original de más de un 10%
#### Comparar con el modelo ComplementNB

Para este apartado se vuelve a hacer uso de GridSearch para obtener los mejores hiperparámetros posibles:

In [200]:
# Instanciar modelo
com_naive_bayes_clf = ComplementNB()

# Definición del conjunto de hiperparámetros
param_grid = {
    'alpha': np.arange(0.01, 1, 0.05),
    'norm': [True, False],
    'fit_prior': [True, False]
    }

# Instanciamos y ejecutamos gridSearch
grid_search = GridSearchCV(com_naive_bayes_clf, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(new_X_train, y_train)

# Obtenemos el mejor hiperparámetro y reentrenamos el modelo
best_params = grid_search.best_params_
print("Hierparámetros más efectivos:", best_params)

com_naive_bayes_clf = ComplementNB(**best_params)
com_naive_bayes_clf.fit(new_X_train, y_train)

# Recalculamos el score
y_pred = com_naive_bayes_clf.predict(new_X_test)
f1_score(y_test, y_pred, average='macro')

Hierparámetros más efectivos: {'alpha': 0.16000000000000003, 'fit_prior': True, 'norm': True}


0.7090320040742759

Se puede observar que el modelo ComplementNB supone una mejoría de un 2.3% en comparación al modelo multinomial

### Ejercicio 3 - Transposicion y similaridad entre palabras

In [201]:
# Transposicion termino-documento + vocabulario
X_train_transposed = new_X_train.T
vocabulary = new_vect.get_feature_names_out()

# Elección de 5 palabras de forma no aleatoria:
# (primero las obtenemos de forma aleatoria para comprobar si son interesantes)
# 98793 -> persuasive
# 127084 -> supreme
# 127746 -> symmetry
# 79184 -> lithium battery
# 62909 -> headache
# ids = random.sample(range(0, X_train_transposed.shape[0]), 5)
ids = [98793, 127084, 127746, 79184, 62909]
selected_words = [vocabulary[i] for i in ids]
print("Índices seleccionados:", ids)
print("Palabras seleccionadas:", selected_words)

# Calcular array de similaridades
cos_sim_arr = cosine_similarity(X_train_transposed[ids], X_train_transposed)

# Recorremos la lista de terminos para obtener los otros cinco mas similares
for i, j in enumerate(ids):
    # Motramos la palabra:
    print(f'Termino numero {i+1}: "{selected_words[i]}"')

    # Obtenemos los indices de los terminos más similares (sin contarse a él mismo)
    # Ordenamos, invertimos el orden (mayor a menor) y obtenemos los 5 primeros indices saltandose el primero (mismo término)
    cos_sim = cos_sim_arr[i]
    sim_indexes = cos_sim.argsort()[::-1][1:6]

    # Mostramos los terminos más similares
    for k,l in enumerate(sim_indexes):
        cos_value = cos_sim[l]
        print(f'Termino similar numero {k+1}: "{vocabulary[l]}"')
        print(f"Similaridad : {cos_value}\n")
    print("-----------------------\n")

Índices seleccionados: [98793, 127084, 127746, 79184, 62909]
Palabras seleccionadas: ['persuasive', 'supreme', 'symmetry', 'lithium battery', 'headache']
Termino numero 1: "persuasive"
Termino similar numero 1: "spurn"
Similaridad : 0.5572896227087307

Termino similar numero 2: "negates"
Similaridad : 0.5397104736765972

Termino similar numero 3: "yes agree"
Similaridad : 0.5347139357541766

Termino similar numero 4: "god reveal"
Similaridad : 0.48597447049215803

Termino similar numero 5: "think conflict"
Similaridad : 0.482322046480582

-----------------------

Termino numero 2: "supreme"
Termino similar numero 1: "supreme court"
Similaridad : 0.7374191586418898

Termino similar numero 2: "supreme power"
Similaridad : 0.4404715164654267

Termino similar numero 3: "court"
Similaridad : 0.35537108325758643

Termino similar numero 4: "eyed"
Similaridad : 0.3517748212863505

Termino similar numero 5: "ruler"
Similaridad : 0.3409303784607524

-----------------------

Termino numero 3: "sy

Aunque en algunos casos los términos no guardan demasiada similaridad, podemos observar como se relacionan con palabras similares o com composiciones de la misma palabra (Ya que estoy utilizando el vectorizador mejorado, el cual utiliza unigramas y digramas).